In [35]:
import pandas as pd
import numpy as np
import easydict

from matplotlib import pyplot as plt

params = easydict.EasyDict({
        'env':'gym-2d-perception-v2',
        'gaze_method':'LookAhead',
        'render':False,
        'dt':0.1,
        'map_scale':10,
        'map_size':[640,480],
        'agent_number':10,
        'agent_max_speed':30,
        'agent_radius':10,
        'drone_max_speed':20,
        'drone_max_acceleration':20,
        'drone_radius':5,
        'drone_max_yaw_speed':80,
        'drone_view_depth' : 80,
        'drone_view_range': 90,
        'record': False,
        'record_img':False,
        'pillar_number':5,
        'max_steps':800
    })



In [41]:
df = pd.read_csv('../experiment/results_1.csv')
df.groupby('Method').mean()
# df['Density'] = (df['Number of agents']*params.agent_radius**2*3.14 + df['Number of pillars']*40**2*3.14) / (params.map_size[0]*params.map_size[1])
# df['Success rate'] = df['Success'] / (df['Success'] + df['Static Collision'] + df['Dynamic Collision'])

,Number of agents,Number of pillars,View depth,View range,Agent speed,Drone speed,Success,Static Collision,Dynamic Collision
Method,,,,,,,,,
LookAhead,7.5,12.5,80.0,90.0,25.0,30.0,147.625,1.333333,21.833333
NoControl,7.5,12.5,80.0,360.0,25.0,30.0,83.625,0.875000,6.583333
Oxford,7.5,12.5,80.0,90.0,25.0,30.0,85.125,25.958333,21.333333


In [40]:
df
# df[(df['Number of agents']==5) & (df['Number of pillars']==10)]
# df.T.plot(kind='bar', title='From Mon To Fri')
# plt.show()

,Method,Number of agents,Number of pillars,View depth,View range,Agent speed,Drone speed,Success,Static Collision,Dynamic Collision
0,Oxford,5,10,80,90,20,30,165,42,22
1,Oxford,5,10,80,90,20,40,204,43,23
2,Oxford,5,10,80,90,30,30,115,26,29
3,LookAhead,5,10,80,90,20,30,176,0,9
4,LookAhead,5,10,80,90,20,40,223,3,15
...,...,...,...,...,...,...,...,...,...,...
67,NoControl,10,15,80,360,30,20,66,0,10
68,Oxford,10,15,80,90,30,30,64,27,37
69,NoControl,10,15,80,360,30,30,80,1,13
70,Oxford,10,15,80,90,30,40,70,46,34
